In [1]:
# Importar librerías necesarias
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
# Cargar los datos
orders = pd.read_csv('../data/orders.csv')
order_products_prior = pd.read_csv('../data/order_products__prior.csv')
products = pd.read_csv('../data/products.csv')
aisles = pd.read_csv('../data/aisles.csv')
departments = pd.read_csv('../data/departments.csv')

In [92]:
departments["department"]

0              frozen
1               other
2              bakery
3             produce
4             alcohol
5       international
6           beverages
7                pets
8     dry goods pasta
9                bulk
10      personal care
11       meat seafood
12             pantry
13          breakfast
14       canned goods
15         dairy eggs
16          household
17             babies
18             snacks
19               deli
20            missing
Name: department, dtype: object

Paso 1: Preparar los datos para un sistema colaborativo basado en usuarios

✅ Objetivo del paso

Crear una matriz de usuario-producto con información de si el usuario ha comprado un producto o no. Esta será la base del sistema de recomendación colaborativo.

In [4]:
'''
Usamos eval_set == 'prior' porque en el dataset de Instacart, la columna eval_set indica a qué conjunto de datos pertenece cada pedido. Tiene 3 valores posibles:
| `eval_set` | Significado                                                                                 |
| ---------- | ------------------------------------------------------------------------------------------- |
| `'prior'`  | Pedidos **anteriores**, usados para entrenamiento. Contienen historial completo de compras. |
| `'train'`  | El último pedido de algunos usuarios, usado como etiqueta para entrenamiento.               |
| `'test'`   | El último pedido de otros usuarios, sin productos (para predicción).                        |
¿Por qué usamos solo prior?
Porque estamos haciendo un modelo colaborativo basado en usuarios, y para eso necesitamos el historial de compras completo, es decir, los productos que los usuarios compraron anteriormente.
Los pedidos en eval_set == 'prior' representan todo el historial de compras antes del último pedido de cada usuario, y es lo que realmente te permite aprender los patrones de comportamiento.
Los pedidos en 'train' y 'test' los dejamos para más adelante, cuando queramos evaluar o predecir.
'''
# 1. Filtra solo los pedidos 'prior'
prior_orders = orders[orders['eval_set'] == 'prior']


In [41]:
prior_orders

,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order
0,2539329,1,prior,1,2,8,NaN
1,2398795,1,prior,2,3,7,15.0
2,473747,1,prior,3,3,12,21.0
3,2254736,1,prior,4,4,7,29.0
4,431534,1,prior,5,4,15,28.0
...,...,...,...,...,...,...,...
3421077,2558525,206209,prior,9,4,15,22.0
3421078,2266710,206209,prior,10,5,18,29.0
3421079,1854736,206209,prior,11,4,10,30.0
3421080,626363,206209,prior,12,1,12,18.0


In [42]:

# 2. Mergeo con order_products_prior, para tener la info de quién compra qué
prior_merged = pd.merge(prior_orders[['order_id', 'user_id']], 
                        order_products_prior[['order_id', 'product_id']], 
                        on='order_id')


In [43]:
prior_merged

,order_id,user_id,product_id
0,2539329,1,196
1,2539329,1,14084
2,2539329,1,12427
3,2539329,1,26088
4,2539329,1,26405
...,...,...,...
32434484,2977660,206209,14197
32434485,2977660,206209,38730
32434486,2977660,206209,31477
32434487,2977660,206209,6567


In [ ]:

# 3. Crea una tabla de interacciones usuario-producto
user_product_interactions = prior_merged.groupby(['user_id', 'product_id']).size().reset_index(name='count')




In [46]:
user_product_interactions

,user_id,product_id,count,interaction,user_idx,product_idx
0,1,196,10,1,0,0
1,1,10258,9,1,0,1
2,1,10326,1,1,0,2
3,1,12427,10,1,0,3
4,1,13032,3,1,0,4
...,...,...,...,...,...,...
13307948,206209,43961,3,1,206208,145
13307949,206209,44325,1,1,206208,6285
13307950,206209,48370,1,1,206208,4737
13307951,206209,48697,1,1,206208,448


Crear matriz dispersa (sparse matrix)
---------
Esto convierte la información de usuarios-productos en una matriz donde cada celda representa una interacción.
Es ideal para trabajar con millones de filas sin consumir demasiada memoria.

In [53]:
from scipy.sparse import csr_matrix

rows = user_product_interactions['user_id'].values
cols = user_product_interactions['product_id'].values
data = user_product_interactions['count'].values  # O puedes usar np.ones(len(rows)) si prefieres binario

sparse_matrix = csr_matrix((data, (rows, cols)))
sparse_matrix


<Compressed Sparse Row sparse matrix of dtype 'int64'
	with 13307953 stored elements and shape (206210, 49689)>

In [ ]:
sparse_matrix[1, 196] # sparse_matrix[i, j] = número de veces que el usuario i compró el producto j. Usado para checkear que todo está bien.

10

Entrenar modelo KNN (User-Based Collaborative Filtering)
------------
Usaremos KNN (K-Nearest Neighbors) para encontrar usuarios similares.
Este modelo no predice productos directamente, sino que permite buscar vecinos similares a un usuario.

In [ ]:
from sklearn.neighbors import NearestNeighbors

busco_vecinos_similares_knn_model = NearestNeighbors(
    metric='cosine',     # Similitud del coseno es común en sistemas de recomendación
    algorithm='brute',   # Funciona bien con matrices dispersas grandes
    n_neighbors=10,      # Número de vecinos (usuarios similares) a buscar
    n_jobs=-1            # Usar todos los núcleos disponibles
)

busco_vecinos_similares_knn_model.fit(sparse_matrix)

'''
¿Qué hace esto?
El modelo se entrena sobre la matriz usuario-producto.

Después podrás pedirle:
👉 “Dame los 10 usuarios más parecidos al usuario 42”
👉 “Dime qué productos compraron que el usuario 42 no ha comprado aún”
'''


NearestNeighbors(algorithm='brute', metric='cosine', n_jobs=-1, n_neighbors=10)

Generar recomendaciones para un usuario usando el modelo KNN
---------
Ahora que el modelo KNN ya está entrenado con la sparse_matrix, vamos a:

-Elegir un usuario.

-Buscar otros usuarios parecidos.

-Ver qué productos compraron esos usuarios que el usuario original no ha comprado aún.

-Recomendarle esos productos.

In [82]:
# Por ejemplo: recomendar productos para el usuario 42
user_id = 1200

distances, indices = busco_vecinos_similares_knn_model.kneighbors(sparse_matrix[user_id], n_neighbors=10)

# Vamos a recomendar productos que compraron los usuarios similares
similar_users = indices[0][1:]  # quitamos el propio usuario

# Encuentra todos los productos que han comprado esos usuarios similares
similar_users_products = sparse_matrix[similar_users].nonzero()[1]

# Contamos cuántas veces aparece cada producto
from collections import Counter
product_counts = Counter(similar_users_products)

# Ordenamos los productos por frecuencia y elegimos los más comunes
recommended_products = [prod for prod, count in product_counts.most_common(10)]


In [83]:
recommended_products

[21288, 43352, 21137, 16797, 6184, 13176, 12341, 39275, 3265, 25588]

In [84]:
recommended_df = products[products['product_id'].isin(recommended_products)]
print(recommended_df[['product_id', 'product_name']])


       product_id                   product_name
3264         3265  Mini Seedless Watermelon Pack
6183         6184                    Clementines
12340       12341                  Hass Avocados
13175       13176         Bag of Organic Bananas
16796       16797                   Strawberries
21136       21137           Organic Strawberries
21287       21288                   Blackberries
25587       25588      Seedless Small Watermelon
39274       39275            Organic Blueberries
43351       43352                    Raspberries
